<a href="https://colab.research.google.com/github/machinelearnear/nelson-openai-master-plan/blob/main/whisperx_transcribir_podcast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🧉 machinelearnear [📺](https://www.youtube.com/c/machinelearnear)

### instalar requerimientos

In [1]:
!pip install git+https://github.com/m-bain/whisperx.git;
!python3 -m pip install -U yt-dlp;

### bajarse el podcast de youtube

In [ ]:
import json
import yt_dlp

URL = 'https://www.youtube.com/watch?v=OA1biHKSyTw'

# ℹ️ See help(yt_dlp.YoutubeDL) for a list of available options and public functions
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(URL, download=False)

    # # ℹ️ ydl.sanitize_info makes the info json-serializable
    # print(json.dumps(ydl.sanitize_info(info)))

In [ ]:
import re

def convert_title(title):
    # Extract the episode number using a regex pattern
    episode_number = re.search(r'#(\d+)', title)
    if episode_number:
        episode_number = episode_number.group(1).zfill(3)
    else:
        raise ValueError("Episode number not found in the input string")

    # Remove any non-alphanumeric characters and split the words into a list
    words = re.findall(r'\b\w+\b', title)

    # Find the index of the hyphen (-) separator
    separator_index = title.find('-')

    if separator_index == -1:
        raise ValueError("Separator (-) not found in the input string")

    # Extract the names after the separator and remove any leading/trailing whitespace
    names = title[separator_index+1:].strip()

    # Combine the episode number and names into the desired format
    result = f"episode_{episode_number}_{names.replace(' ', '_')}".lower()
    return result

input_title = ydl.sanitize_info(info)["title"]
output_title = convert_title(input_title)
print(output_title)  # Output: episode_048_alejandro_dolina

In [ ]:
ydl_opts = {
    'outtmpl': f'output/{output_title}',
    'format': 'm4a/bestaudio/best',
    # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
    'postprocessors': [{  # Extract audio using ffmpeg
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'wav',
    }]
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URL)

In [ ]:
# youtube_video = "https://www.youtube.com/watch?v=NJYQzHVz-xI&t=513s&ab_channel=machinelearnear"

In [2]:
# !python -m yt_dlp --output "archivo.%(ext)s" --extract-audio --audio-format wav $youtube_video

### ahora si ... correr whisperx

In [4]:
# hf_token = "<aca-pone-tu-hf-token" # https://huggingface.co/settings/tokens

In [ ]:
def extract_guest_name(title):
    # Find the index of the hyphen (-) separator
    separator_index = title.find('-')

    if separator_index == -1:
        raise ValueError("Separator (-) not found in the input string")

    # Extract the guest's name after the separator and remove any leading/trailing whitespace
    guest_name = title[separator_index+1:].strip()

    return guest_name

input_title = "El Método Rebord #48 - Alejandro Dolina"
guest_name = extract_guest_name(input_title)
print(guest_name)  # Output: Alejandro Dolina

#### transcripción en español

In [5]:
# !whisperx archivo.wav --model large-v2 --language es --diarize

#### traducción de español a inglés

In [6]:
# !whisperx archivo.wav --hf_token $hf_token --model large-v2 --language es --task translate --vad_filter --align_model WAV2VEC2_ASR_LARGE_LV60K_960H 

#### transcripción en español con VAD

In [ ]:
# !whisperx archivo.wav --hf_token $hf_token --model large-v2 --language es --vad_filter --align_model WAV2VEC2_ASR_LARGE_LV60K_960H 